In [1]:
import fastavro
from sqlalchemy import create_engine, MetaData, Table, select
import datetime

In [3]:
date_backup=datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
engine = create_engine('mysql+pymysql://root@localhost/flaskmysql')
metadata = MetaData()
table = Table('jobs', metadata, autoload_with=engine)
query = select(table)

In [5]:
with engine.connect() as conn, open('jobs.avro'+date_backup, 'wb') as out:
    result = conn.execute(query)
    schema = {
    'type': 'record',
    'name': 'Job',
    'fields': [
        {'name': 'id', 'type': 'int'},
        {'name': 'job', 'type': 'string'},
    ],
    }
    results_as_dict = result.mappings().all()
    fastavro.writer(out, schema, results_as_dict)